# Sentiment Analysis with an RNN

In this notebook, you'll implement a recurrent neural network that performs sentiment analysis. 
>Using an RNN rather than a strictly feedforward network is more accurate since we can include information about the *sequence* of words. 

Here we'll use a dataset of movie reviews, accompanied by sentiment labels: positive or negative.

<img src="assets/reviews_ex.png" width=40%>

### Network Architecture

The architecture for this network is shown below.

<img src="assets/network_diagram.png" width=40%>

>**First, we'll pass in words to an embedding layer.** We need an embedding layer because we have tens of thousands of words, so we'll need a more efficient representation for our input data than one-hot encoded vectors. You should have seen this before from the Word2Vec lesson. You can actually train an embedding with the Skip-gram Word2Vec model and use those embeddings as input, here. However, it's good enough to just have an embedding layer and let the network learn a different embedding table on its own. *In this case, the embedding layer is for dimensionality reduction, rather than for learning semantic representations.*

>**After input words are passed to an embedding layer, the new embeddings will be passed to LSTM cells.** The LSTM cells will add *recurrent* connections to the network and give us the ability to include information about the *sequence* of words in the movie review data. 

>**Finally, the LSTM outputs will go to a sigmoid output layer.** We're using a sigmoid function because positive and negative = 1 and 0, respectively, and a sigmoid will output predicted, sentiment values between 0-1. 

We don't care about the sigmoid outputs except for the **very last one**; we can ignore the rest. We'll calculate the loss by comparing the output at the last time step and the training label (pos or neg).

---
### Load in and visualize the data

In [1]:
import numpy as np

# read data from text files
with open('data/reviews.txt', 'r') as f:
    reviews = f.read()
with open('data/labels.txt', 'r') as f:
    labels = f.read()

In [2]:
print(reviews[:1000])
print()
print(labels[:30])

bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   
story of a man who has unnatural feelings for a pig . starts out with a opening scene that is a terrific example of absurd comedy . a formal orchestra audience is turn

## Data pre-processing

The first step when building a neural network model is getting your data into the proper form to feed into the network. Since we're using embedding layers, we'll need to encode each word with an integer. We'll also want to clean it up a bit.

You can see an example of the reviews data above. Here are the processing steps, we'll want to take:
>* We'll want to get rid of periods and extraneous punctuation.
* Also, you might notice that the reviews are delimited with newline characters `\n`. To deal with those, I'm going to split the text into each review using `\n` as the delimiter. 
* Then I can combined all the reviews back together into one big string.

First, let's remove all punctuation. Then get all the text without the newlines and split it into individual words.

In [51]:
from string import punctuation

print(punctuation)

# get rid of punctuation
reviews = reviews.lower() # lowercase, standardize
all_text = ''.join([c for c in reviews if c not in punctuation])


!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [52]:
# split by new lines and spaces
reviews_split = all_text.split('\n') #get a list with an element for every review
print("{} reviews total".format(len(reviews_split)))

all_text = ' '.join(reviews_split) #get a string of all words separated by single space
print("all_text length: {}".format(len(all_text)))

# create a list of words
words = all_text.split()
print("{} words in all reviews".format(len(words)))

25001 reviews total
all_text length: 33351075
6020196 words in all reviews


In [53]:
words[:10]

['bromwell', 'high', 'is', 'a', 'cartoon', 'comedy', 'it', 'ran', 'at', 'the']

### Encoding the words

The embedding lookup requires that we pass in integers to our network. The easiest way to do this is to create dictionaries that map the words in the vocabulary to integers. Then we can convert each of our reviews into integers so they can be passed into the network.

> **Exercise:** Now you're going to encode the words with integers. Build a dictionary that maps words to integers. Later we're going to pad our input vectors with zeros, so make sure the integers **start at 1, not 0**.
> Also, convert the reviews to integers and store the reviews in a new list called `reviews_ints`. 

In [59]:
# feel free to use this import 
from collections import Counter

def create_lookup_tables(words):
    """
    Create lookup tables for vocabulary
    :param words: Input list of words
    :return: Two dictionaries, vocab_to_int, int_to_vocab
    """
    word_counts = Counter(words)
    # sorting the words from most to least frequent in text occurrence
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    # create int_to_vocab dictionaries
    int_to_vocab = {(ii+1): word for ii, word in enumerate(sorted_vocab)}
    vocab_to_int = {word: (ii) for ii, word in int_to_vocab.items()}

    return vocab_to_int, int_to_vocab

## Build a dictionary that maps words to integers
vocab_to_int_dict, int_to_vocab_dict = create_lookup_tables(words)
# print(vocab_to_int_dict['the'])
# print(int_to_vocab_dict[1])

## use the dict to tokenize each review in reviews_split
## store the tokenized reviews in reviews_ints
def convert_to_ints(reviews_list):
    '''
    given a 2-dim array, convert words to ints
    '''
    reviews_ints = []
    for review in reviews_list:
        this_review_ints = []
        review_words = review.split(' ')
        for review_word in review_words:
            if review_word in vocab_to_int_dict.keys(): 
                this_review_ints.append(vocab_to_int_dict[review_word])
        reviews_ints.append(this_review_ints)
        
    return reviews_ints

reviews_ints = convert_to_ints(reviews_split)

print("number of reviews in reviews_ints:")
print(len(reviews_ints))

number of reviews in reviews_ints:
25001


**Test your code**

As a text that you've implemented the dictionary correctly, print out the number of unique words in your vocabulary and the contents of the first, tokenized review.

In [7]:
# stats about vocabulary
print('Unique words: ', len((vocab_to_int_dict)))  # should ~ 74000+
print()

# print tokens in first review
print('Tokenized review: \n', reviews_ints[:1])

Unique words:  74072

Tokenized review: 
 [[21025, 308, 6, 3, 1050, 207, 8, 2138, 32, 1, 171, 57, 15, 49, 81, 5785, 44, 382, 110, 140, 15, 5194, 60, 154, 9, 1, 4975, 5852, 475, 71, 5, 260, 12, 21025, 308, 13, 1978, 6, 74, 2395, 5, 613, 73, 6, 5194, 1, 24103, 5, 1983, 10166, 1, 5786, 1499, 36, 51, 66, 204, 145, 67, 1199, 5194, 19869, 1, 37442, 4, 1, 221, 883, 31, 2988, 71, 4, 1, 5787, 10, 686, 2, 67, 1499, 54, 10, 216, 1, 383, 9, 62, 3, 1406, 3686, 783, 5, 3483, 180, 1, 382, 10, 1212, 13583, 32, 308, 3, 349, 341, 2913, 10, 143, 127, 5, 7690, 30, 4, 129, 5194, 1406, 2326, 5, 21025, 308, 10, 528, 12, 109, 1448, 4, 60, 543, 102, 12, 21025, 308, 6, 227, 4146, 48, 3, 2211, 12, 8, 215, 23]]


### Encoding the labels

Our labels are "positive" or "negative". To use these labels in our network, we need to convert them to 0 and 1.

> **Exercise:** Convert labels from `positive` and `negative` to 1 and 0, respectively, and place those in a new list, `encoded_labels`.

In [8]:
# 1=positive, 0=negative label conversion
encoded_labels = []

labels_split = labels.strip().split('\n')

for label in labels_split:
    if label == 'positive':
        encoded_labels.append(1)
    elif label == 'negative':
        encoded_labels.append(0)
    else:
        print("bad label: {}".format(label))

encoded_labels = np.asarray(encoded_labels)
                      
print("encoded_labels: ")
print(encoded_labels[:10])
    

encoded_labels: 
[1 0 1 0 1 0 1 0 1 0]


### Removing Outliers

As an additional pre-processing step, we want to make sure that our reviews are in good shape for standard processing. That is, our network will expect a standard input text size, and so, we'll want to shape our reviews into a specific length. We'll approach this task in two main steps:

1. Getting rid of extremely long or short reviews; the outliers
2. Padding/truncating the remaining data so that we have reviews of the same length.

<img src="assets/outliers_padding_ex.png" width=40%>

Before we pad our review text, we should check for reviews of extremely short or long lengths; outliers that may mess with our training.

In [9]:
# outlier review stats
review_lens = Counter([len(x) for x in reviews_ints])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))

Zero-length reviews: 1
Maximum review length: 2514


Okay, a couple issues here. We seem to have one review with zero length. And, the maximum review length is way too many steps for our RNN. We'll have to remove any super short reviews and truncate super long reviews. This removes outliers and should allow our model to train more efficiently.

> **Exercise:** First, remove *any* reviews with zero length from the `reviews_ints` list and their corresponding label in `encoded_labels`.

In [10]:
print('Number of reviews before removing outliers: ', len(reviews_ints))
print('Number of labels before removing outliers: ', len(encoded_labels))

## remove any reviews/labels with zero length from the reviews_ints list.
counter = 0
for review in reviews_ints:
    if len(review) == 0:
#        print("Zero length review found at index: {}".format(counter))
        #reviews_ints.remove(review)
        #encoded_labels.remove(encoded_label)
        del reviews_ints[counter]
        if len(encoded_labels) > counter:
            del encoded_labels[counter]
        break
#     else:
#         print("Review length found at index: {} = {}".format(counter, len(review)))
    counter += 1

#reviews_ints = None
#encoded_labels = None

print('Number of reviews after removing outliers: ', len(reviews_ints))
print('Number of labels after removing outliers: ', len(encoded_labels))

Number of reviews before removing outliers:  25001
Number of labels before removing outliers:  25000
Number of reviews after removing outliers:  25000
Number of labels after removing outliers:  25000


---
## Padding sequences

To deal with both short and very long reviews, we'll pad or truncate all our reviews to a specific length. For reviews shorter than some `seq_length`, we'll pad with 0s. For reviews longer than `seq_length`, we can truncate them to the first `seq_length` words. A good `seq_length`, in this case, is 200.

> **Exercise:** Define a function that returns an array `features` that contains the padded data, of a standard size, that we'll pass to the network. 
* The data should come from `review_ints`, since we want to feed integers to the network. 
* Each row should be `seq_length` elements long. 
* For reviews shorter than `seq_length` words, **left pad** with 0s. That is, if the review is `['best', 'movie', 'ever']`, `[117, 18, 128]` as integers, the row will look like `[0, 0, 0, ..., 0, 117, 18, 128]`. 
* For reviews longer than `seq_length`, use only the first `seq_length` words as the feature vector.

As a small example, if the `seq_length=10` and an input review is: 
```
[117, 18, 128]
```
The resultant, padded sequence should be: 

```
[0, 0, 0, 0, 0, 0, 0, 117, 18, 128]
```

**Your final `features` array should be a 2D array, with as many rows as there are reviews, and as many columns as the specified `seq_length`.**

This isn't trivial and there are a bunch of ways to do this. But, if you're going to be building your own deep learning networks, you're going to have to get used to preparing your data.

In [11]:
def pad_features(reviews_ints, seq_length):
    ''' Return features of review_ints, where each review is padded with 0's 
        or truncated to the input seq_length.
    '''
    features = np.zeros((len(reviews_ints), seq_length), dtype=np.int64)
    
    row_counter = 0
    for this_review_ints in reviews_ints:
        if len(this_review_ints) > seq_length:
            #replace whole seq_length length array with beginning of review.
            this_review_ints = this_review_ints[:seq_length]
            features[row_counter] = this_review_ints
        elif len(this_review_ints) < seq_length:
            #replace end of array with ints, leaving beginning zeros
            replacement_indices = np.arange((seq_length-len(this_review_ints)), seq_length)
            np.put(features[row_counter], replacement_indices, this_review_ints)    
        row_counter += 1
    
    return features

In [12]:
# Test your implementation!

seq_length = 200

features = pad_features(reviews_ints, seq_length=seq_length)

## test statements - do not change - ##
assert len(features)==len(reviews_ints), "Your features should have as many rows as reviews."
assert len(features[0])==seq_length, "Each feature row should contain seq_length values."

# print first 10 values of the first 30 batches 
print(features[:50,:12])
#print(features[:10])

[[    0     0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0     0]
 [22382    42 46418    15   706 17139  3389    47    77    35  1819    16]
 [ 4505   505    15     3  3342   162  8312  1652     6  4819    56    17]
 [    0     0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0     0]
 [   54    10    14   116    60   798   552    71   364     5     1   730]
 [    0     0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0     0]
 [    1   330   578    34

## Training, Validation, Test

With our data in nice shape, we'll split it into training, validation, and test sets.

> **Exercise:** Create the training, validation, and test sets. 
* You'll need to create sets for the features and the labels, `train_x` and `train_y`, for example. 
* Define a split fraction, `split_frac` as the fraction of data to **keep** in the training set. Usually this is set to 0.8 or 0.9. 
* Whatever data is left will be split in half to create the validation and *testing* data.

In [13]:
split_frac = 0.8

## split data into training, validation, and test data (features and labels, x and y)
train_split_index = int(len(features)*split_frac)
valid_split_frac = split_frac + (1-split_frac)/2
valid_split_index = int(len(features)*valid_split_frac)

train_x = features[:train_split_index]
train_y = encoded_labels[:train_split_index]

val_x = features[train_split_index : valid_split_index]
val_y = encoded_labels[train_split_index : valid_split_index]

test_x = features[valid_split_index:]
test_y = encoded_labels[valid_split_index:]

## print out the shapes of your resultant feature data
print("train_x.shape: {}".format(train_x.shape))
print("train_y: {}".format(train_y.shape))

print("val_x.shape: {}".format(val_x.shape))
print("val_y: {}".format(val_y.shape))

print("test_x.shape: {}".format(test_x.shape))
print("test_y: {}".format(test_y.shape))

# print(train_x[0])
# print(valid_x[0])
# print(test_x[0])

train_x.shape: (20000, 200)
train_y: (20000,)
val_x.shape: (2500, 200)
val_y: (2500,)
test_x.shape: (2500, 200)
test_y: (2500,)


**Check your work**

With train, validation, and test fractions equal to 0.8, 0.1, 0.1, respectively, the final, feature data shapes should look like:
```
                    Feature Shapes:
Train set: 		 (20000, 200) 
Validation set: 	(2500, 200) 
Test set: 		  (2500, 200)
```

---
## DataLoaders and Batching

After creating training, test, and validation data, we can create DataLoaders for this data by following two steps:
1. Create a known format for accessing our data, using [TensorDataset](https://pytorch.org/docs/stable/data.html#) which takes in an input set of data and a target set of data with the same first dimension, and creates a dataset.
2. Create DataLoaders and batch our training, validation, and test Tensor datasets.

```
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
train_loader = DataLoader(train_data, batch_size=batch_size)
```

This is an alternative to creating a generator function for batching our data into full batches.

In [14]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

# dataloaders
batch_size = 50

# make sure to SHUFFLE your data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [15]:
# obtain one batch of training data
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([50, 200])
Sample input: 
 tensor([[   10,   329,    49,  ...,    10,    68,  1032],
        [    0,     0,     0,  ...,  1738,   160,  3754],
        [    0,     0,     0,  ...,     3,    45,     4],
        ...,
        [    9,     1,  3261,  ...,  2121, 21703,    13],
        [    0,     0,     0,  ...,   115,    12,    20],
        [    0,     0,     0,  ...,    28,   124,   108]])

Sample label size:  torch.Size([50])
Sample label: 
 tensor([0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1,
        0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0,
        1, 0])


---
# Sentiment Network with PyTorch

Below is where you'll define the network.

<img src="assets/network_diagram.png" width=40%>

The layers are as follows:
1. An [embedding layer](https://pytorch.org/docs/stable/nn.html#embedding) that converts our word tokens (integers) into embeddings of a specific size.
2. An [LSTM layer](https://pytorch.org/docs/stable/nn.html#lstm) defined by a hidden_state size and number of layers
3. A fully-connected output layer that maps the LSTM layer outputs to a desired output_size
4. A sigmoid activation layer which turns all outputs into a value 0-1; return **only the last sigmoid output** as the output of this network.

### The Embedding Layer

We need to add an [embedding layer](https://pytorch.org/docs/stable/nn.html#embedding) because there are 74000+ words in our vocabulary. It is massively inefficient to one-hot encode that many classes. So, instead of one-hot encoding, we can have an embedding layer and use that layer as a lookup table. You could train an embedding layer using Word2Vec, then load it here. But, it's fine to just make a new layer, using it for only dimensionality reduction, and let the network learn the weights.


### The LSTM Layer(s)

We'll create an [LSTM](https://pytorch.org/docs/stable/nn.html#lstm) to use in our recurrent network, which takes in an input_size, a hidden_dim, a number of layers, a dropout probability (for dropout between multiple layers), and a batch_first parameter.

Most of the time, you're network will have better performance with more layers; between 2-3. Adding more layers allows the network to learn really complex relationships. 

> **Exercise:** Complete the `__init__`, `forward`, and `init_hidden` functions for the SentimentRNN model class.

Note: `init_hidden` should initialize the hidden and cell state of an lstm layer to all zeros, and move those state to GPU, if available.

In [16]:
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


In [17]:
import torch.nn as nn

class SentimentRNN(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super(SentimentRNN, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # define all layers
        #The Embedding Layer
        self.embed = nn.Embedding(vocab_size, embedding_dim)
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        
        #self.fc_out = nn.Linear(self.hidden_dim*200, 1)  #200 is seq length
        self.fc_out = nn.Linear(self.hidden_dim, output_size)  #200 is seq length
        self.sig = nn.Sigmoid()
        
        self.drop = nn.Dropout(drop_prob)
        
    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = x.size(0)
        
        x = self.embed(x)
        #x = self.embed_out(x)
        
        r_out, hidden = self.lstm(x, hidden)
        
        # shape output to be (batch_size*seq_length, hidden_dim)
        out = r_out.contiguous().view(-1, self.hidden_dim)  #200 is seq length  
        
        # return last sigmoid output and hidden state
        out = self.drop(out)
        out = self.fc_out(out)
        out = self.sig(out)      
        out = out.view(batch_size, -1)  # reshape to be batch_size first
        out = out[:, -1] ##### get last batch of labels
            
        return out, hidden
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

## Instantiate the network

Here, we'll instantiate the network. First up, defining the hyperparameters.

* `vocab_size`: Size of our vocabulary or the range of values for our input, word tokens.
* `output_size`: Size of our desired output; the number of class scores we want to output (pos/neg).
* `embedding_dim`: Number of columns in the embedding lookup table; size of our embeddings.
* `hidden_dim`: Number of units in the hidden layers of our LSTM cells. Usually larger is better performance wise. Common values are 128, 256, 512, etc.
* `n_layers`: Number of LSTM layers in the network. Typically between 1-3

> **Exercise:** Define the model  hyperparameters.


In [23]:
# Instantiate the model w/ hyperparams
vocab_size = len((vocab_to_int_dict))
output_size = 1
embedding_dim = 350
hidden_dim = 256
n_layers = 3

net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)

print(net)

SentimentRNN(
  (embed): Embedding(74072, 350)
  (lstm): LSTM(350, 256, num_layers=3, batch_first=True, dropout=0.5)
  (fc_out): Linear(in_features=256, out_features=1, bias=True)
  (sig): Sigmoid()
  (drop): Dropout(p=0.5)
)


---
## Training

Below is the typical training code. If you want to do this yourself, feel free to delete all this code and implement it yourself. You can also add code to save a model by name.

>We'll also be using a new kind of cross entropy loss, which is designed to work with a single Sigmoid output. [BCELoss](https://pytorch.org/docs/stable/nn.html#bceloss), or **Binary Cross Entropy Loss**, applies cross entropy loss to a single value between 0 and 1.

We also have some data and training hyparameters:

* `lr`: Learning rate for our optimizer.
* `epochs`: Number of times to iterate through the training dataset.
* `clip`: The maximum gradient value to clip at (to prevent exploding gradients).

In [24]:
# loss and optimization functions
lr=0.001

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)


In [25]:
# training params

epochs = 4 # 3-4 is approx where I noticed the validation loss stop decreasing

counter = 0
print_every = 100
clip=5 # gradient clipping

# move model to GPU, if available
if(train_on_gpu):
    net.cuda()

net.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)

    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        output, h = net(inputs, h)
        
        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()

                output, val_h = net(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

Epoch: 1/4... Step: 100... Loss: 0.642161... Val Loss: 0.644621
Epoch: 1/4... Step: 200... Loss: 0.676787... Val Loss: 0.758379
Epoch: 1/4... Step: 300... Loss: 0.690638... Val Loss: 0.796227
Epoch: 1/4... Step: 400... Loss: 0.679672... Val Loss: 0.763233
Epoch: 2/4... Step: 500... Loss: 0.696036... Val Loss: 0.693501
Epoch: 2/4... Step: 600... Loss: 0.673910... Val Loss: 0.638476
Epoch: 2/4... Step: 700... Loss: 0.478105... Val Loss: 0.595141
Epoch: 2/4... Step: 800... Loss: 0.486245... Val Loss: 0.492825
Epoch: 3/4... Step: 900... Loss: 0.345836... Val Loss: 0.510441
Epoch: 3/4... Step: 1000... Loss: 0.547915... Val Loss: 0.486132
Epoch: 3/4... Step: 1100... Loss: 0.286541... Val Loss: 0.456971
Epoch: 3/4... Step: 1200... Loss: 0.190345... Val Loss: 0.428146
Epoch: 4/4... Step: 1300... Loss: 0.195519... Val Loss: 0.419700
Epoch: 4/4... Step: 1400... Loss: 0.302920... Val Loss: 0.428639
Epoch: 4/4... Step: 1500... Loss: 0.223418... Val Loss: 0.416196
Epoch: 4/4... Step: 1600... Loss: 

In [26]:
#Save the model
torch.save(net.state_dict(), "sentiment_rnn_model.pt")


---
## Testing

There are a few ways to test your network.

* **Test data performance:** First, we'll see how our trained model performs on all of our defined test_data, above. We'll calculate the average loss and accuracy over the test data.

* **Inference on user-generated data:** Second, we'll see if we can input just one example review at a time (without a label), and see what the trained model predicts. Looking at new, user input data like this, and predicting an output label, is called **inference**.

In [27]:
# Get test data loss and accuracy

net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
net.load_state_dict(torch.load("sentiment_rnn_model.pt"))

if(train_on_gpu):
    net.cuda()

test_losses = [] # track loss
num_correct = 0
batch_counter = 0
batch_quit = 10
total_preditions = 0

# init hidden state
h = net.init_hidden(batch_size)

net.eval()
# iterate over test data
for inputs, labels in test_loader:
    batch_counter += 1
    if batch_counter > batch_quit:
        break  #it crashes if I run through the whole test.  I don't know why.
    
    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in h])

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    
    # get predicted outputs
    output, h = net(inputs, h)
    #print(output)
    total_preditions += len(output)
    
    # calculate loss
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze())  # rounds to the nearest integer
    
    # compare predictions to true label
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)
    
    if batch_counter > 10:
        break


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
#test_acc = num_correct/len(test_loader.dataset)
test_acc = num_correct/total_preditions
print("Test accuracy: {:.3f}".format(test_acc))

tensor([0.0192, 0.9810, 0.0254, 0.1150, 0.9825, 0.8391, 0.0243, 0.9806, 0.7436,
        0.9602, 0.0413, 0.0197, 0.9686, 0.6163, 0.5175, 0.2300, 0.7885, 0.0222,
        0.9783, 0.0649, 0.9778, 0.9549, 0.9804, 0.0977, 0.9803, 0.9753, 0.9599,
        0.9792, 0.1119, 0.1136, 0.0596, 0.9547, 0.4037, 0.9808, 0.9767, 0.0674,
        0.9703, 0.3140, 0.8970, 0.0829, 0.7580, 0.0276, 0.9544, 0.9784, 0.3167,
        0.0353, 0.6987, 0.9810, 0.9754, 0.1635], device='cuda:0',
       grad_fn=<SelectBackward>)
tensor([0.9810, 0.7193, 0.3671, 0.9760, 0.1673, 0.9699, 0.9777, 0.4151, 0.2200,
        0.9162, 0.0232, 0.0272, 0.0809, 0.8683, 0.9539, 0.6616, 0.9705, 0.9822,
        0.0203, 0.0848, 0.0771, 0.1431, 0.9811, 0.0258, 0.0200, 0.3686, 0.0222,
        0.9769, 0.0215, 0.9794, 0.9782, 0.0267, 0.9814, 0.0217, 0.0280, 0.0813,
        0.9640, 0.9805, 0.9787, 0.6643, 0.1265, 0.1118, 0.9755, 0.9772, 0.9698,
        0.8676, 0.4058, 0.2215, 0.9795, 0.0341], device='cuda:0',
       grad_fn=<SelectBackward>)
te

### Inference on a test review

You can change this test_review to any text that you want. Read it and think: is it pos or neg? Then see if your model predicts correctly!
    
> **Exercise:** Write a `predict` function that takes in a trained net, a plain text_review, and a sequence length, and prints out a custom statement for a positive or negative review!
* You can use any functions that you've already defined or define any helper functions you want to complete `predict`, but it should just take in a trained net, a text review, and a sequence length.


In [82]:
# negative test review
test_review_neg = 'The worst movie I have seen; acting was terrible and I want my money back. This movie had bad acting and the dialogue was slow.'


In [125]:
def preprocess_review(review_string, sequence_length=200):
    review_string_no_punctuation = ''.join([c for c in review_string if c not in punctuation])
    review_list = []
    review_list.append(review_string_no_punctuation)
    test_review_ints = convert_to_ints(review_list)
    test_review_ints = pad_features(test_review_ints, sequence_length)
    return test_review_ints
    

def predict(net, test_review, sequence_length=200):
    ''' Prints out whether a give review is predicted to be 
        positive or negative in sentiment, using a trained model.
        
        params:
        net - A trained net 
        test_review - a review made of normal text and punctuation
        sequence_length - the padded length of a review
        '''
    test_batch_size = len(test_review)
    
    use_gpu = torch.cuda.is_available()
    
    if use_gpu:
        net.cuda()
    else:
        net.cpu()
    
    h = net.init_hidden(test_batch_size)
    input_tensor = torch.from_numpy(test_review)
    
    if use_gpu:
        input_tensor = input_tensor.cuda()
    
    net.eval()

    output, h = net(input_tensor, h)
    
    return output.cpu().detach().numpy()
    
    
    
# print custom response based on whether test_review is pos/neg


net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
net.load_state_dict(torch.load("sentiment_rnn_model.pt"))

def predict_movie(review_text):
    test_review_list = preprocess_review(review_text)
    prediction = predict(net, test_review_list)

    if prediction < 0.5:
        print("this was a negative review ({:.3f}):".format(prediction[0]))
    else:
        print("this was a positive review ({:.3f}):".format(prediction[0]))
    print(review_text)
    


In [128]:
predict_movie(test_review_neg)

# positive test review
#test_review_pos = 'This movie had the best acting and the dialogue was so good. I loved it.'
test_review_pos = 'This movie was the most fun I had in a long time. The action was terriffic. The actress made me laugh.'

predict_movie(test_review_pos)

this was a negative review (0.022):
The worst movie I have seen; acting was terrible and I want my money back. This movie had bad acting and the dialogue was slow.
this was a positive review (0.974):
This movie was the most fun I had in a long time. The action was terriffic. The actress made me laugh.


### Try out test_reviews of your own!

Now that you have a trained model and a predict function, you can pass in _any_ kind of text and this model will predict whether the text has a positive or negative sentiment. Push this model to its limits and try to find what words it associates with positive or negative.

Later, you'll learn how to deploy a model like this to a production environment so that it can respond to any kind of user data put into a web app!

In [127]:
test_review = '<<insert movie review text here>>'
predict_movie(test_review)



this was a positive review (0.802):
<<insert movie review text here>>
